# Final Project Submission

* Student name: Shayan Abdul Karim Khan
* Student pace: Self Paced
* Scheduled project review date/time:
* Instructor name: Abhineet Kulkarni

## Problem Overview

Each year more than 42,000 people are needlessly killed on American streets and thousands more are injured. We call this suffering traffic “accidents” — but, in reality, we have the power to prevent traffic collisions. To counter these fatalities and injuries, the Department of Transportations (DOTs) across the US are working towards a new vision for safety called "Vision Zero". 

Vision Zero recognizes that people will sometimes make mistakes, so the road system and related policies should be designed to ensure those inevitable mistakes do not result in severe injuries or fatalities. This means that system designers and policymakers are expected to improve the roadway environment, policies (such as speed management), and other related systems to lessen the severity of crashes by working with stakeholders ranging from auto manufacturers to policy makers.

Similarly, the Chicago DOT wants to conduct a study to understand the underlying patterns for car accidents. The ask is to use the data available to predict the primary contributory cause of a car accident, given information about the car, the people in the car, the road conditions etc. 

## Business Questions

In its work towards Vision Zero, the Chicago DOT has the following questions:

1. What are the common variables between the different classes of traffic accidents?
<br>

2. Is there a discernable pattern to the accidents?


### Resolution Approach

To help inform appropriate measures towards the movement to Vision Zero, aa classifier for predictions. The data will be assessed for recommendations on 4 different fronts:
<br>

1. Type of Vehicle: For example; Passenger Vehilce, Commercial Vehicle, etc.
<br>

2. Weather conditions: For example; Rainy, Snow, Windy, etc.
<br>

3. Locations of accidents: This is to understand which locations are prone to accidents
<br>

4. Type of accidents: This is important to understand the cause behind it. For eexample; a DUI accident would have drastically different preventative measures as compared to a sober over-speeding accident
<br>

For prioritization purposes, it will also bee critical to understand what accidents resullted in fatalities or injuries and which didn't.

## Data Sources

The data being used is from the Chicago Polic Department (CPD) and is provided oninee by the City of Chicago. The data is from 2015 to the present day. It is updated daily but the data being used as a part of this project is recent until 3rd February 2023. 

There are 3 datasets being used for analysis:

1. Traffic crashes data summary: https://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3ifhttps://data.cityofchicago.org/Transportation/Traffic-Crashes-Crashes/85ca-t3ifv 
<br>

2. Vehicles information involved in a crash: https://data.cityofchicago.org/Transportation/Traffic-Crashes-Vehicles/68nd-jvt3 
<br>

3. People information involved in a crash: https://data.cityofchicago.org/Transportation/Traffic-Crashes-People/u6pd-qa9d 

Considering that this is the official dataset of the City of Chicago regarding traffic crashes, it can be easilly concluded that it would be the most reliable dataset available. Using this dataset will also increase the confidence of the client on the results and recommendations of this project. 

## Data Understanding

### Importing the Data

### Dataset 1

### Dataset 2

### Entity Relationship Diagram

### Data Limitations

### Avenues of Analysis

### Summary

## Data Preparation

### Data Cleaning

### Data Processing

## Modeling and Evaluation

### Baseline Model

### KNN Model

### Random Forests Model

### XG Boost Model

### Models Comaprison

## Final Model Evlauation

## Conclusion

### Improvements

### Next Steps

### Recommendations